<a href="https://colab.research.google.com/github/GouriNayak/DEMO/blob/main/Weatherforecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the required libraries


In [9]:
import numpy as np  # linear algebra
import pandas as pd # Data processing , CSV file I/O (e.g. pd.read_csv)

Load the csv file as data frame

In [11]:
df=pd.read_csv("https://raw.githubusercontent.com/GouriNayak/DEMO/main/weatherAUS.csv")

In [12]:
print('size of weather data frame is :',df.shape)

size of weather data frame is : (142193, 24)


Display data

In [13]:
print(df[0:5])

         Date Location  MinTemp  ...  RainToday  RISK_MM  RainTomorrow
0  2008-12-01   Albury     13.4  ...         No      0.0            No
1  2008-12-02   Albury      7.4  ...         No      0.0            No
2  2008-12-03   Albury     12.9  ...         No      0.0            No
3  2008-12-04   Albury      9.2  ...         No      1.0            No
4  2008-12-05   Albury     17.5  ...         No      0.2            No

[5 rows x 24 columns]


Checking null values

In [14]:
print(df.count().sort_values())

Sunshine          74377
Evaporation       81350
Cloud3pm          85099
Cloud9am          88536
Pressure9am      128179
Pressure3pm      128212
WindDir9am       132180
WindGustDir      132863
WindGustSpeed    132923
WindDir3pm       138415
Humidity3pm      138583
Temp3pm          139467
WindSpeed3pm     139563
Humidity9am      140419
RainToday        140787
Rainfall         140787
WindSpeed9am     140845
Temp9am          141289
MinTemp          141556
MaxTemp          141871
Date             142193
Location         142193
RISK_MM          142193
RainTomorrow     142193
dtype: int64


As we can see the first 4 coloumns have less than 60% data, we can ignore these 4 colur

we don't need the location column because we are going to find if it will rain in Australia(not lacation specific)

we are going to drop the date column too.

we need to remove RISK_MM because we want to predict 'RainTomorrow' and 

RISK_MM can leak
we remove unwanted data
'Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','RISK_MM','Date'





In [15]:
df = df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','RISK_MM','Date'],axis=1)

In [16]:
print(df.shape)

(142193, 17)


# Let us get rid of all null values in df

In [17]:
df = df.dropna(how='any')

In [18]:
print(df.shape)

(112925, 17)


its time to remove the outliers in our data - we are using z-score to detect and remove the 

In [19]:
from scipy import stats

In [21]:
z=np.abs(stats.zscore(df._get_numeric_data()))

In [22]:
print(z)

[[0.11756741 0.10822071 0.20666127 ... 1.14245477 0.08843526 0.04787026]
 [0.84180219 0.20684494 0.27640495 ... 1.04184813 0.04122846 0.31776848]
 [0.03761995 0.29277194 0.27640495 ... 0.91249673 0.55672435 0.15688743]
 ...
 [1.44940294 0.23548728 0.27640495 ... 0.58223051 1.03257127 0.34701958]
 [1.16159206 0.46462594 0.27640495 ... 0.25166583 0.78080166 0.58102838]
 [0.77784422 0.4789471  0.27640495 ... 0.2085487  0.37167606 0.56640283]]


In [23]:
df=df[(z<3).all(axis=1)]

In [24]:
print(df.shape)

(107868, 17)


lets deal with the categorical cloumns now
Simply change yes/no to 1/0 for RainToDay and RainTomorrow

In [34]:
df['RainToday'].replace(('no:0','yes:1'),inplace = True)

In [35]:
df['RainTomorrow'].replace(('no:0','yes:1'),inplace = True)

see unique values and convert them to int using pd.getDummies()

In [36]:
categorical_columns = ['WindGustDir','WindDir3pm','WindDir9am']

In [37]:
for col in categorical_columns:
	print(np.unique(df[col]))

['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']


transform the categorical columns

In [38]:
df=pd.get_dummies(df,columns=categorical_columns)

next step is to standardize our data using MinMaxScaler

In [40]:
from sklearn import preprocessing

In [46]:
scaler = preprocessing.MinMaxScaler()
